In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

#créer une session dans le master
spark = SparkSession.builder.master("spark://172.20.53.96:7077").appName("WDC-readAll").getOrCreate()
#spark = SparkSession.builder.master("local").appName("WDC-readAll").getOrCreate()

#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = 'bbd95ea3c1174caa88345404b84e458f'
aws_secret_access_key = 'eaf2a72ecf9845f583af7f3513c44f25'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/27 15:12:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/27 15:12:19 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [5]:
readall = spark.read.option("header",True) \
  .csv("s3a://test-out/wdc-httpswww/**")

readall.take(5)

import pyspark.sql.functions as f
csall=readall.groupby("pset").agg(f.sum("count").alias('count')).sort(f.desc("count"))
csall.show(truncate=0)
csall.createOrReplaceTempView("CSET")

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|pset                                                                                                                                                                 |count       |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|<schema.org/item>|<schema.org/position>|isa:<schema.org/ListItem>                                                                                                    |6.82470722E8|
|<schema.org/itemListElement>|isa:<schema.org/BreadcrumbList>                                                                                                         |4.56613881E8|
|<schema.org/item>|<schema.org/name>|<schema.org/position>|isa:<schema.org/ListItem>           

In [7]:
spark.sql("select pset,count from CSET where pset like '%isa:<schema.org/Person>%'  limit 1000").show(truncate=0)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|pset                                                                                                                                                                                        |count       |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|<schema.org/name>|isa:<schema.org/Person>                                                                                                                                                   |1.55338855E8|
|<schema.org/image>|<schema.org/name>|<schema.org/url>|isa:<schema.org/Person>                                                                                                          